In [ ]:
%run deeponet_pde.py

In [15]:
import numpy as np
from sklearn import gaussian_process as gp

In [22]:
x = np.linspace(0, 1, num=1000)[:, None] # x就是100个0-1之间的点
#print(x)

In [23]:
# random ()
#Generate `n` random feature vectors.
import numpy as np
u = np.random.randn(1000, 10000)
print(u.shape)

(1000, 10000)


In [24]:
length_scale=0.2
K = gp.kernels.RBF(length_scale=length_scale)
K = K(x) 

In [25]:
#print(K)
print("Dimensions of K:", K.shape)


Dimensions of K: (1000, 1000)


In [28]:
#computes the Cholesky decomposition of a matrix that consists of the covariance matrix self.K with a small added diagonal perturbation.
L = np.linalg.cholesky(K + 1e-13 * np.eye(1000))
#print(L)
print(L.shape)

(1000, 1000)


In [29]:
# generating random samples from a Gaussian distribution using the Cholesky decomposition method.
random_result = np.dot(L, u).T # the dot pruduct of L and u
# print(random_result)
print(random_result.shape)

(10000, 1000)


In [30]:
# in system.py
print(1 * np.random.rand(10000)[:, None])

[[0.79027274]
 [0.55496262]
 [0.09202611]
 ...
 [0.11977756]
 [0.57337878]
 [0.26581102]]


In [3]:
import numpy as np
from scipy import interpolate

# Define a class with the eval_u_one method
class FunctionInterpolator:
    def __init__(self, interp_type="linear"):
        self.interp = interp_type
        # Assume self.x is a 1D array of x-coordinates for your data points
        self.x = np.array([1, 2, 3, 4, 5])
    
    def eval_u_one(self, y, x):
        if self.interp == "linear":
            return np.interp(x, np.ravel(self.x), y)
        f = interpolate.interp1d(
            np.ravel(self.x), y, kind=self.interp, copy=False, assume_sorted=True
        )
        return f(x)

# Create an instance of the class
interpolator = FunctionInterpolator()

# Define a function (represented as data points)
# This function has data points corresponding to the x-coordinates in self.x
function_data = np.array([2, 4, 3, 6, 5])  # Replace with your data points

# Define a specific point where you want to compute the function value
specific_point = [2.5, 3.5]  # Replace with your desired x-coordinate

# Call the eval_u_one method to compute the function value at the specific point
function_value = interpolator.eval_u_one(function_data, specific_point)

# Print the computed function value
print(f"Function value at x = {specific_point}: {function_value}")


Function value at x = [2.5, 3.5]: [3.5 4.5]


In [1]:
import numpy as np

# Define a class with the eval_u method
class FunctionInterpolator:
    def __init__(self, interp_type="linear"):
        self.interp = interp_type
        # Assume self.x is a 1D array of x-coordinates for your data points
        self.x = np.array([1, 2, 3, 4, 5])
    
    def eval_u(self, ys, sensors):
        if self.interp == "linear":
            return np.vstack([np.interp(sensors, np.ravel(self.x), y).T for y in ys])

# Create an instance of the class
interpolator = FunctionInterpolator()

# Define a list of functions (represented as data points)
# Each function has data points corresponding to the x-coordinates in self.x
function1 = np.array([2, 4, 3, 6, 5])  # Replace with your data points
function2 = np.array([1, 3, 1.5, 4.5, 3])  # Replace with your data points
function3 = np.array([0, 2, 1, 3, 2.5])  # Replace with your data points

# Define sensor points where you want to interpolate function values
sensor_points = np.array([1.5, 2.5, 3.5])  # Replace with your desired sensor points

# Call the eval_u method to interpolate function values at sensor points
interpolated_values = interpolator.eval_u([function1, function2, function3], sensor_points)

# Print the interpolated values
print("Interpolated Values:")
print(interpolated_values)


Interpolated Values:
[[3.   3.5  4.5 ]
 [2.   2.25 3.  ]
 [1.   1.5  2.  ]]


In [4]:
from scipy.integrate import solve_ivp

# Define the ODE system as a Python function
def odesystem(t, y):
    dydt = -2 * t * y
    return dydt

# Define the initial conditions
t_span = (0, 5)
y0 = [1.0]

# Solve the ODEs using the 'RK45' method
solution = solve_ivp(odesystem, t_span, y0, method='RK45', t_eval=[0, 1, 2, 3, 4, 5])

# Access the solution at specific time points
print("Time points:", solution.t)
print("Solution values:", solution.y[0])

Time points: [0 1 2 3 4 5]
Solution values: [1.00000000e+00 3.67998883e-01 1.83178607e-02 1.23961050e-04
 1.55675581e-07 1.46443319e-07]


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import itertools

import numpy as np
import tensorflow as tf

import deepxde as dde
from spaces import FinitePowerSeries, FiniteChebyshev, GRF
from system import LTSystem, ODESystem, DRSystem, CVCSystem, ADVDSystem
from utils import merge_values, trim_to_65535, mean_squared_error_outlier, safe_test



2023-09-02 23:33:13.136580: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 23:33:13.816509: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-09-02 23:33:13.818002: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-02 23:33:17.270122: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
Using backend: tensorflow.compat.v1
Other supported backends: tensorflow, pytorch, jax, paddle.
paddle supports more examples now and is recommended.


Instructions for updating:
non-resource variables are not supported in the long term


2023-09-02 23:33:24.307947: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1960] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [3]:
def ode_system(T):
    """ODE"""

    def g(s, u, x):
        # Antiderivative
        return u
        # Nonlinear ODE
        # return -s**2 + u
        # Gravity pendulum
        # k = 1
        # return [s[1], - k * np.sin(s[0]) + u]

    s0 = [0]
    # s0 = [0, 0]  # Gravity pendulum
    return ODESystem(g, s0, T)


In [4]:
problem = "ode"
T = 1

m = 100
num_train = 10000
num_test = 100000
lr = 0.001
epochs = 50000

system = ode_system(T)
space = GRF(1, length_scale=0.2, N=1000, interp="cubic")

X_train, y_train = system.gen_operator_data(space, m, num_train)
print("X_train:",X_train)
print("y_train:",y_train)

Generating operator data...
'gen_operator_data' took 37.114730 s

X_train: [array([[-0.67377417, -0.7164547 , -0.75806802, ...,  0.72740741,
         0.70937354,  0.69228458],
       [-0.71306527, -0.67376013, -0.63586445, ...,  1.61989663,
         1.67443211,  1.7253884 ],
       [-1.27192265, -1.21020951, -1.14291121, ..., -1.7065931 ,
        -1.72820423, -1.75077834],
       ...,
       [-0.24678339, -0.22648383, -0.21078069, ..., -0.31240173,
        -0.30215867, -0.28251597],
       [-1.13882236, -1.07704478, -1.0113834 , ...,  0.97439072,
         0.97543754,  0.98001548],
       [-0.19130821, -0.26006125, -0.33023965, ...,  0.82079097,
         0.93020328,  1.03678781]]), array([[0.97407125],
       [0.50855862],
       [0.58565644],
       ...,
       [0.68308219],
       [0.28086793],
       [0.34745995]])]
y_train: [[ 0.2708296 ]
 [ 0.08726522]
 [ 0.15243661]
 ...
 [ 0.60232189]
 [ 0.00510228]
 [-0.26626907]]


In [5]:
print(X_train[0].shape)
print(X_train[1].shape)

(10000, 100)
(10000, 1)


In [ ]:

# Network
nn = "opnn"
activation = "relu"
initializer = "Glorot normal"  # "He normal" or "Glorot normal"
dim_x = 1 if problem in ["ode", "lt"] else 2
if nn == "opnn":
    net = dde.maps.DeepONet(
        [m, 40, 40], # m indicates the input layer size, and the subsequent 40, 40 indicate the sizes of two hidden layers.
        [dim_x, 40, 40],
        activation,
        initializer,
        use_bias=True,
        stacked=False,
    )
elif nn == "fnn":
    net = dde.maps.FNN([m + dim_x] + [100] * 2 + [1], activation, initializer)
elif nn == "resnet":
    net = dde.maps.ResNet(m + dim_x, 1, 128, 2, activation, initializer)